# Exploring SFPD Police Incident Report Data
The dataset available has SFPD police incident report data from January 2003 to May 2018. The goal is to perform analysis on the available data to discern trends over the years in San Francisco's crime scene.

The data is freely available at [datasf.org](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry).


In [ ]:
# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing as prepro
import folium
from folium.plugins import HeatMap

In [ ]:
# read data from csv file
data = pd.read_csv("./data/SFPD-Incident-Reports-Jan2003-May2018.csv")

#preview rows,cols
#print(data.shape)
data.info()

#preview data
data.head()

There are 35 columns but we definitely don't need them all. It also looks like there may be null values throughout the dataset. Of particular interest is the Date and Time columns which we can use for temporal analysis. We also need to check for nulls and duplicates and drop them from our dataset.

In [ ]:
# lets read the csv data again but this time parse the date and time columns as datetime
data = pd.read_csv("./data/SFPD-Incident-Reports-Jan2003-May2018.csv", parse_dates=[['Date','Time']])

#we don't need all 35 columns, so lets select what we determine to be relevant
data = data[["Date_Time","Category","Descript",'DayOfWeek',"PdDistrict","Resolution","Address","X","Y"]]

#preview rows,cols
data.info()

#preview data
data.head()

In [ ]:
#check for nulls
data.isnull().sum()

In [ ]:
#drop nulls
data.dropna(inplace=True)

data.isnull().sum()

In [ ]:
#check how many duplicate entries there are
data.duplicated().sum()

In [ ]:
#drop duplicate entries
data.drop_duplicates(subset=None, keep="first", inplace=True)

#preview rows,cols
data.info()

#preview data
data.head()

We can infer from the number of rows that there are 2,129,525 number of crimes in the dataset. Of which around 5,520 were duplicates and one had a null value leaving a total of 2,124,004 crimes.

We also have temporal data in the dataset that we can format better.

We can breakdown the Date_Time column to split data by year, month and day. Note that we only have 2018 data from Jan to May so we are going to drop 2018.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['Date_Time']).year
data['Month'] = pd.DatetimeIndex(data['Date_Time']).month
data['Day'] = pd.DatetimeIndex(data['Date_Time']).day

In [ ]:
data = data[data['Year'] != 2018]
data.info()

After dropping the 2018 year data, we are left with 2,079,049 incident reports.

We've cleaned up this data enough to finally start some exploratory data analysis.

We can visualise the data with a correlation heatmap for all our features in the dataset but we would have to first encode our categorical features (PdDistrict, Category, Address,etc.)

In [ ]:
# make a copy of the data
data_enc = data.copy()

# label encoding
for i in ['Category','Descript','DayOfWeek','PdDistrict','Resolution','Address']:
    data_enc[i] = prepro.LabelEncoder().fit_transform(data_enc[i])

In [ ]:
#correlation matrix
corr_mat = data_enc.corr()
corr_mat

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(corr_mat, annot=True, linewidths=.1)
plt.title('Heatmap for dataset', fontsize=16)
plt.show()

We can determine the different categories of crimes and number of crimes in each category.

In [ ]:
num_crimes_cat = data["Category"].value_counts()
print(num_crimes_cat)

It seems that the most prevalent type of crime from 2003 to 2018 is larceny/theft. We can plot a bar chart to visualise the data here.

In [ ]:
plt.figure()
num_crimes_cat.plot(kind='bar',xlabel='Category',ylabel='Occurences',title='Crime occurences by category',figsize=(16,15))

We can also check which day has the highest number of crimes committed.

In [ ]:
crime_days = data["DayOfWeek"].value_counts()
crime_days.plot(kind='bar',xlabel='Day',ylabel='Occurences',title='Crime occurences by day of the week',figsize=(10,10),width=0.75)

We can check the descriptions and see what is the most prevalent as well.

In [ ]:
desc = data["Descript"].value_counts()
# limit to top 20 due to volume of descriptions
desc[:20].plot(kind='bar',xlabel='Description',ylabel='Occurences',title='Crime occurences vs Description', figsize=(10,10))

The PdDistricts column can give us an idea of which neighbourhoods we may want to avoid in SF.

In [ ]:
plt.figure()
data['PdDistrict'].value_counts().plot(kind='bar',xlabel='District',ylabel='Occurences',title='Crime occurences in each District', figsize=(10,10))
plt.show()

In [ ]:
years_data = data['Year'].value_counts().sort_index()
years_data.plot(kind='bar',xlabel='Year',ylabel='Occurences',title='Crime occurences per year', figsize=(10,10))

Suppose we want to find out about a specific category of crime being committed over the years. We can now do so since we processed the year information earlier.

In [ ]:
vehicle_theft_df = data[data['Category'] == 'VEHICLE THEFT']
yearly_vehicle_theft = vehicle_theft_df.groupby('Year').Date_Time.nunique()
plt.figure()
yearly_vehicle_theft.plot(kind='bar',xlabel='Year', ylabel='Occurences of Vehicle Theft', title='Number of Vehicle Thefts per year',figsize=(16,9))
plt.show()
# vehicle_theft_df.head()

In [ ]:
vandalism_df = data[data['Category'] == 'VANDALISM']
yearly_vandalism = vandalism_df.groupby('Year').Date_Time.nunique()
plt.figure()
yearly_vandalism.plot(kind='bar',xlabel='Year', ylabel='Occurences of Vandalism', title='Number of Vandalism reports per year',figsize=(16,9))
plt.show()

We can plot the incident report geo coordinate data to generate a heatmap that visualises where most crimes are reported in the city.

In [ ]:
geo_df = data.copy()
sample_data = data.sample(int(0.1 * len(geo_df)))
X_Y_pairs = list(zip(list(sample_data.Y), list(sample_data.X)))

In [ ]:
map = folium.Map(location=[37.733795, -122.446747],zoom_start=12)
HeatMap(X_Y_pairs).add_to(map)
map